<a href="https://colab.research.google.com/github/slapazromero/EjercicioNumpy/blob/main/Ejercicio_Numpy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from PIL import Image
from random import randint

In [ ]:
class ImagenNumPy:

  @staticmethod
  def crear_imagen(ancho = 100, alto = 100, color = [0, 0, 0], pintar = True):
    data = np.zeros((alto, ancho, len(color)), dtype = np.uint8)
    data[0:alto, 0:ancho] = color
    return ImagenNumPy._pintar_imagen(data, pintar)

  @staticmethod
  def crear_mosaico(elementos_h = 5, elementos_v = 3, ancho = 100, alto = 100, pintar = True):
    for y in range(elementos_v):
      fila = ImagenNumPy.crear_imagen(ancho, alto, (randint(0, 255), randint(0, 255), randint(0, 255)), False)
      
      for x in range(elementos_h - 1):
        fila = np.hstack([fila, ImagenNumPy.crear_imagen(ancho, alto, (randint(0, 255), randint(0, 255), randint(0, 255)), False)])
      
      if 'data' in locals() :
        data = np.vstack([data, fila])
      
      else:
        data = np.vstack([fila,])
    
    return ImagenNumPy._pintar_imagen(data, pintar)

  @staticmethod
  def redimensionar_imagen(data, ancho, alto, pintar = True):
    img = ImagenNumPy._cambiar_formato_image(data)
    img = img.resize((ancho, alto))
    
    return ImagenNumPy._pintar_imagen(img, pintar)

  @staticmethod
  def redimensionar_imagen_ancho(data, nuevo_ancho, pintar = True):
    img = ImagenNumPy._cambiar_formato_image(data)
    img = ImagenNumPy.redimensionar_imagen(data, nuevo_ancho, ImagenNumPy._calcular_altura(data, nuevo_ancho))
    
    return ImagenNumPy._pintar_imagen(img, pintar)

  @staticmethod
  def redimensionar_imagen_alto(data, nuevo_alto, pintar = True):
    img = ImagenNumPy._cambiar_formato_image(data)
    img = ImagenNumPy.redimensionar_imagen(data, ImagenNumPy._calcular_ancho(data, nuevo_alto), nuevo_alto)
    
    return ImagenNumPy._pintar_imagen(img, pintar)

  @staticmethod
  def recortar_trozo(data, ancho, alto, coordenadas = (0, 0), pintar = True):
    coordenadas = ImagenNumPy._validar_coordenadas(data, coordenadas)
    img = ImagenNumPy._cambiar_formato_image(data)
    data = ImagenNumPy._cambiar_formato_np(data)

    if coordenadas[0] >= img.width or coordenadas[1] >= img.height:
      print('Coordenadas superiores a la imagen')
      return ImagenNumPy._pintar_imagen(data, pintar)

    elif coordenadas[0] + ancho >= img.width or coordenadas[1] + alto >= img.height:
      data = data[coordenadas[1] : img.height, coordenadas[0] : img.width]
      return ImagenNumPy._pintar_imagen(data, pintar)

    else:
      data = data[coordenadas[1] : coordenadas[1] + alto, coordenadas[0] : coordenadas[0] + ancho]
      return ImagenNumPy._pintar_imagen(data, pintar)

  @staticmethod
  def apilar_sin_deformar(data1, data2, horizontal = True, pintar = True):
    imagen1 = ImagenNumPy._cambiar_formato_image(data1)
    imagen2 = ImagenNumPy._cambiar_formato_image(data2)
    data1 = ImagenNumPy._cambiar_formato_np(data1)
    data2 = ImagenNumPy._cambiar_formato_np(data2)
    if horizontal:
      if imagen1.height != imagen2.height:
        data2 = ImagenNumPy.redimensionar_imagen_alto(data2, imagen1.height, False)
      
      return ImagenNumPy._pintar_imagen(np.hstack((data1, data2)), pintar)
    else:
      if imagen1.width != imagen2.width:
        data2 = ImagenNumPy.redimensionar_imagen_ancho(data2, imagen1.width, False)
    
    return ImagenNumPy._pintar_imagen(np.vstack((data1, data2)), pintar)

  @staticmethod
  def apilar_deformar(data1, data2, horizontal = True, pintar = True):
    imagen1 = ImagenNumPy._cambiar_formato_image(data1)
    imagen2 = ImagenNumPy._cambiar_formato_image(data2)
    data1 = ImagenNumPy._cambiar_formato_np(data1)
    data2 = ImagenNumPy._cambiar_formato_np(data2)
    if horizontal:
      if imagen1.height != imagen2.height:
        data2 = ImagenNumPy.redimensionar_imagen(data2, imagen2.width, imagen1.height, False)
      
      return ImagenNumPy._pintar_imagen(np.hstack((data1, data2)), pintar)
    else:
      if imagen1.width != imagen2.width:
        data2 = ImagenNumPy.redimensionar_imagen(data2, imagen1.width, imagen2.height, False)
    
    return ImagenNumPy._pintar_imagen(np.vstack((data1, data2)), pintar)

  @staticmethod
  def insertar_imagen(data1, data2, coordenadas = (0, 0), pintar = True):
    coordenadas = ImagenNumPy._validar_coordenadas(data1, coordenadas)
    data1 = np.copy(ImagenNumPy._cambiar_formato_np(data1))
    data2 = ImagenNumPy._cambiar_formato_np(data2)

    x_data1 = coordenadas[0] + data2.shape[1]
    y_data1 = coordenadas[1] + data2.shape[0]

    x_data2 = data1.shape[1] - coordenadas[0]
    y_data2 = data1.shape[0] - coordenadas[1]

    if x_data1 > data1.shape[1]: x_data1 = data1.shape[1]
    if y_data1 > data1.shape[0]: y_data1 = data1.shape[0]

    data1[coordenadas[1]:y_data1,coordenadas[0]:x_data1] = data2[0:y_data2,0:x_data2]
    return ImagenNumPy._pintar_imagen(data1, pintar)

  @staticmethod
  def insertar_imagen_redimensionada(data1, data2, ancho, alto, coordenadas = (0, 0), pintar = True):
    data2 = ImagenNumPy.redimensionar_imagen(data2, alto, ancho, False)
    return ImagenNumPy.insertar_imagen(data1, data2, coordenadas, pintar)
        
  @staticmethod
  def _cambiar_formato_np(data):
    if not isinstance(data, (np.ndarray, np.generic)):
      return np.asarray(data)
    else:
      return data

  @staticmethod
  def _cambiar_formato_image(data):
    if not isinstance(data, (np.ndarray, np.generic)):
      return data
    else:
      return Image.fromarray(data)

  @staticmethod
  def _pintar_imagen(data, pintar = True):
    data = ImagenNumPy._cambiar_formato_np(data)
    if pintar:
      return Image.fromarray(data)
    else:
      return data

  @staticmethod
  def _calcular_altura(data, ancho):
    img = Image.fromarray(data)
    return (img.height * ancho) // img.width

  @staticmethod
  def _calcular_ancho(data, alto):
    img = Image.fromarray(data)
    return (img.width * alto) // img.height

  @staticmethod
  def _validar_coordenadas(img, coordenadas):
    img = ImagenNumPy._cambiar_formato_image(img)
    coor_x = 0
    coor_y = 0
    if coordenadas[0] < 0:
      coor_x = 0
    elif coordenadas[0] > img.width:
      coor_x = img.width
    else:
      coor_x = coordenadas[0]
    if coordenadas[1] < 0:
      coor_y = 0
    elif coordenadas[1] > img.height:
      coor_y = img.height
    else:
      coor_y = coordenadas[1]
    
    return (coor_x, coor_y)